<a href="https://colab.research.google.com/github/anllkvk/-irket-i-RAG-Projesi/blob/main/RagChatbotColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-genai sentence-transformers faiss-cpu PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.1 MB/s eta 0:00:00


In [10]:
import os
from getpass import getpass

# Gemini API anahtarı gir
gemini_key = getpass("Gemini API key (Google AI Studio'dan aldığın): ")
if gemini_key:
    os.environ["GEMINI_API_KEY"] = gemini_key

# HuggingFace token gir
hf_key = getpass("HuggingFace token (https://huggingface.co/settings/tokens): ")
if hf_key:
    os.environ["HF_TOKEN"] = hf_key

print("🔑 API anahtarları ayarlandı (gizli).")


Gemini API key (Google AI Studio'dan aldığın): ··········
HuggingFace token (https://huggingface.co/settings/tokens): ··········
🔑 API anahtarları ayarlandı (gizli).


In [3]:
!mkdir -p /content/docs
!wget -q -O /content/docs/is_kanunu.pdf "https://cdn.istanbul.edu.tr/FileHandler2.ashx?f=is-kanunu-%284857%29.pdf"


In [4]:
import PyPDF2

def read_text_from_file(path):
    text = ""
    if path.endswith(".pdf"):
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() or ""
    else:
        with open(path, "r", encoding="utf-8") as f:
            text = f.read()
    return text

doc_text = read_text_from_file("/content/docs/is_kanunu.pdf")
print(doc_text[:500])  # ilk 500 karakter


İŞ KANUNU  
 
Kanun  Numarası  : 4857  
Kabul  Tarihi  : 22/5/2003  
Yayımlandığı  Resmî  Gazete  : Tarih   : 10/6/2003  Sayı : 25134   
Yayımlandığı  Düstur  : Tertip  : 5 Cilt : 42  
 
BİRİNCİ  BÖLÜM  
Genel  Hükümler  
Amaç  ve kapsam  
Madde  1 - Bu Kanunun  amacı  işverenler  ile bir iş sözleşmesine  dayanarak  çalıştırılan  
işçilerin  çalışma  şartları  ve çalışma  ortamına  ilişkin  hak ve sorumluluklarını  düzenlemektir.  
Bu Kanun,  4 üncü  Maddedeki  istisnalar  dışında  kalan  bütün 


In [5]:
def chunk_text(text, max_len=500):
    words = text.split()
    chunks, current = [], []
    for w in words:
        current.append(w)
        if len(current) >= max_len:
            chunks.append(" ".join(current))
            current = []
    if current:
        chunks.append(" ".join(current))
    return chunks

chunks = chunk_text(doc_text, max_len=300)
print("Toplam parça:", len(chunks))
print(chunks[0][:200])  # ilk chunk


Toplam parça: 63
İŞ KANUNU Kanun Numarası : 4857 Kabul Tarihi : 22/5/2003 Yayımlandığı Resmî Gazete : Tarih : 10/6/2003 Sayı : 25134 Yayımlandığı Düstur : Tertip : 5 Cilt : 42 BİRİNCİ BÖLÜM Genel Hükümler Amaç ve kaps


In [6]:
from sentence_transformers import SentenceTransformer

EMB_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embedder = SentenceTransformer(EMB_MODEL)

X = embedder.encode(chunks, batch_size=32, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
import faiss
import pickle

d = X.shape[1]
index = faiss.IndexFlatIP(d)
index.add(X)

faiss.write_index(index, "/content/faiss.index")
with open("/content/chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

print("📦 FAISS index oluşturuldu. Vektör sayısı:", index.ntotal)


📦 FAISS index oluşturuldu. Vektör sayısı: 63


In [8]:
def search(query, k=3):
    qv = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(qv, k)
    return [(chunks[i], float(D[0][j])) for j, i in enumerate(I[0])]

print(search("işçi hakları nelerdir?", 2))


[('Kanunun 32 nci maddesiyle; bu bentte yer alan “Mevzuattan veya sözleşmeden doğan haklarını takip” ibaresinden sonra gelmek üzere “veya yükümlülüklerini yerine getirmek” ibaresi eklenmiştir. Hakkındaki iddialara karşı savunmasını almadan bir işçinin belirsiz süreli iş sözleşmesi, o işçinin davranışı veya verimi ile ilgili nedenlerle feshedilemez. Ancak, işverenin 25 inci maddenin (II) numaralı bendi şartlar ına uygun fesih hakkı saklıdır. Fesih bildirimine itiraz ve usulü Madde 20 – (Değişik birinci fıkra: 12/10/2017 -7036/11 md.) İş sözleşmesi feshedilen işçi, fesih bildiriminde sebep gösterilmediği veya gösterilen sebebin geçerli bir sebep olmadığı iddias ı ile fesih bildiriminin tebliği tarihinden itibaren bir ay içinde işe iade talebiyle, İş Mahkemeleri Kanunu hükümleri uyarınca arabulucuya başvurmak zorundadır. Arabuluculuk faaliyeti sonunda anlaşmaya varılamaması hâlinde, son tutanağın düzenlendiği tarihten itibaren, iki hafta içinde iş mahkemesinde dava açılabilir. Taraflar an

In [12]:
from google import genai

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

def rag_chat(query, k=3):
    # ilgili parçaları getir
    results = search(query, k)
    context = "\n\n".join([r[0] for r in results])

    prompt = f"""
Sen bir yardımcı chatbot'sun.
Kullanıcının sorusunu yanıtlamak için aşağıdaki bağlamı kullan:
---
{context}
---
Soru: {query}
Cevap:
    """

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )
    return response.text

print(rag_chat("Fazla mesai nasıl hesaplanır?"))


Sağlanan bağlama göre fazla mesai şu şekilde hesaplanır:

*   **Normal Fazla Çalışma:** Haftalık kırkbeş saati aşan çalışmalar fazla çalışmadır. Bu çalışmalarda, her bir saat fazla çalışma için verilecek ücret, normal çalışma ücretinin saat başına düşen miktarının **yüzde elli yükseltilmesiyle** ödenir.
*   **Fazla Sürelerle Çalışma:** Haftalık çalışma süresinin sözleşmelerle kırkbeş saatin altında belirlendiği durumlarda, ortalama haftalık çalışma süresini aşan ve kırkbeş saate kadar yapılan çalışmalar fazla sürelerle çalışmadır. Bu çalışmalarda, her bir saat fazla çalışma için verilecek ücret normal çalışma ücretinin saat başına düşen miktarının **yüzde yirmibeş yükseltilmesiyle** ödenir.
*   **Serbest Zaman Kullanımı:** İşçi, fazla çalışma veya fazla sürelerle çalışma karşılığı zamlı ücret yerine isterse, fazla çalıştığı her saat karşılığında **bir saat otuz dakikayı**, fazla sürelerle çalıştığı her saat karşılığında **bir saat onbeş dakikayı** serbest zaman olarak kullanabilir. İşç

In [13]:
import gradio as gr

# RAG Chat fonksiyonu
def rag_chat_with_history(query, chat_history):
    """
    chat_history: önceki (soru, cevap) listesi
    """
    # En iyi 3 parçayı getir (proje için uygun k=3)
    k = 3
    results = search(query, k)
    context = "\n\n".join([r[0] for r in results])

    prompt = f"""
Sen bir yardımcı chatbot'sun.
Kullanıcının sorusunu yanıtlamak için aşağıdaki bağlamı kullan:
---
{context}
---
Soru: {query}
Cevap:
    """

    # Gemini API çağrısı
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    ).text

    # chat_history güncelle
    chat_history = chat_history or []
    chat_history.append((query, response))
    return chat_history, chat_history


In [15]:
with gr.Blocks() as demo:
    gr.Markdown("## 🟢 Şirket İçi Bilgi Bankası Chatbot")

    chatbot = gr.Chatbot(label="Chatbot")
    user_input = gr.Textbox(label="Soru", placeholder="Sorunuzu yazın...")
    clear_btn = gr.Button("Temizle")

    # input gönderildiğinde chat fonksiyonu çalışsın
    user_input.submit(
        rag_chat_with_history,
        inputs=[user_input, chatbot],
        outputs=[chatbot, chatbot]
    )

    clear_btn.click(lambda: [], None, chatbot)

demo.launch()


/tmp/ipython-input-1567283328.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chatbot")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eae2240e8676649b67.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
